## Task 3: Game bot (50%)
Choose one of the games in OpenAI gym and make a bot that play it using GA.

Beside the code, add a link to a video of your bot playing.
##### Hint: There are many ways to do this, one of which is making a simple ANN and instead of using gradient decent use GA mutation and cross over to update your network.

In [1]:
import random
import time
from typing import List

#making bot for Open AI gym game using ANN and GA to update the weights of network
import gym as gym
import numpy as np
import tensorflow as tf

from gym import Env
from gym.spaces import Discrete, Box
import cv2
import matplotlib.pyplot as plt

In [2]:
#!pip install gym[atari]
#!pip install gym[accept-rom-license]

In [3]:
env = gym.make('ALE/AirRaid-v5', render_mode='human', obs_type="grayscale")

C:\Users\SunagatullinAyaz\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (250, 160)
  logger.warn(


In [4]:
#pip uninstall gym atari-py ale-py
#pip install gym[atari]

In [5]:
# CNN model for deciding what action to take according to observation
# output = 0 -> noop
# output = 1 -> fire
# output = 2 -> right
# output = 3 -> left
# output = 4 -> rightfire
# output = 5 -> leftfire

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])


In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model.build(input_shape=(None, 250, 160, 1))

In [8]:
model.build()

In [9]:
number_of_weights = model.count_params()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 158, 8)       80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 79, 8)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 77, 16)       1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 38, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 36, 16)        2320      
                                                                 
 flatten (Flatten)           (None, 33984)             0

In [10]:
#.layers[1].get_weights()[0].shape, model.layers[1].get_weights()[1].shape

In [11]:
model.layers[7].get_weights()[0].shape, model.layers[7].get_weights()[1].shape

((12, 6), (6,))

In [12]:
# # these weights are found by GA (code below)
# best_weights = [0.1823618174305801,
#                 0.7713836499876504,
#                 0.6511879077319767,
#                 0.8187707680405115,
#                 0.29137521060185034]
#
# # play with the best weights, our goal is 500 steps.
# env = gym.make('CartPole-v1', render_mode='human')
# first_layer_weights = np.array(best_weights[:4]).reshape(4, 1)
# first_layer_bias = np.array(best_weights[4:5])
# model.layers[0].set_weights([first_layer_weights, first_layer_bias])
# observation = env.reset()[0]
# done = False
# action_count = 0
# total_reward = 0
# while not done and action_count < 500:
#     action = model.predict(np.array([observation]), verbose=0)[0][0]
#     action = round(action)
#     #print('action  ', action)
#     observation, reward, truncated, terminated, info = env.step(action)
#     action_count += 1
#     done = terminated or truncated
#     total_reward += reward
#     #time.sleep(0.1)
#     env.render()
#
# print(f"Total reward: {total_reward}")
# env.close()

# LEARNING PROCESS

In [13]:
#range of values for weights
# Initial range was [-1, 1].
# After some testing and learning the problem,
# I found that [0, 1] is better range, just to make learning faster.
LOW = -1
UP = 1

In [14]:
import numpy.polynomial.polynomial as poly

def create_weight() -> float:
    return random.uniform(LOW, UP)

In [15]:
def create_individual() -> List[float]:
    return [create_weight() for _ in range(number_of_weights)]

def create_population() -> List[List[float]]:
    return [create_individual() for _ in range(population_size)]

In [16]:
def crossover(parent1: List[float], parent2: List[float]) -> (List[float], List[float]):
    crossover_point = random.randint(0, number_of_weights - 1)
    if random.random() < p_crossover:
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    else:
        return parent1, parent2

In [17]:
def selection(population: List[List[float]]) -> (List[float], List[float]):
    return random.choices(
        population=population,
        weights=[fitness(individual) for individual in population],
        k=2)

In [18]:
def mutation(individual) -> List[float]:
    for i in range(number_of_weights):
        if random.random() < p_mutation:
            individual[i] = create_weight()
    return individual

In [19]:
def run_ga(pop_size: int, gen_limit: int, p_crossover: float, p_mutation: float) -> List[float]:
    print("Creating initial population...")
    population_main = create_population()
    print("Evaluating initial population...")
    population_main = sorted(population_main, key=fitness, reverse=True)
    print("Main population sorted.")
    print(f"Best fitness, GEN INIT : {fitness(population_main[0])}")

    for generation in range(gen_limit):
        population_next = population_main[:2]

        print("Creating next population...")
        for _ in range(pop_size // 2 - 1):
            parent1, parent2 = selection(population_main)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutation(child1)
            child2 = mutation(child2)
            population_next.append(child1)
            population_next.append(child2)
        population_main = population_next
        print("Evaluating next population...")
        population_main = sorted(population_main, key=fitness, reverse=True)
        print("Next population sorted.")
        best_f = fitness(population_main[0])
        if best_f == 500:
            print(f"Best fitness, GEN {generation} : {best_f}")
            return population_main[0]
        else:
            print(f"Best fitness, GEN {generation} : {best_f}")

    population_main = sorted(population_main, key=fitness, reverse=True)

    return population_main[0]

In [20]:
# Fitness is equal to total reward in our case,
# so to determine reward we need to play the game each time
# actions are decided by NN
def fitness(individual: List[float]) -> float:
    inserted = 0
    for i, layer in enumerate(model.layers):
        if i in [1, 3, 5]:
            continue

        layer_weights_shape = layer.get_weights()[0].shape
        layer_biases_shape = layer.get_weights()[1].shape

        # total number of weights in layer = product of all dimensions
        total_n_weights = np.prod(layer_weights_shape)
        total_n_biases = np.prod(layer_biases_shape)

        # get weights and biases from individual
        layer_weights = np.array(individual[inserted:inserted + total_n_weights]).reshape(layer_weights_shape)
        layer_biases = np.array(
            individual[inserted + total_n_weights:inserted + total_n_weights + total_n_biases]).reshape(
            layer_biases_shape)

        # set weights and biases in model
        layer.set_weights([layer_weights, layer_biases])

        # update inserted
        inserted += total_n_weights + total_n_biases

    observation = env.reset()[0]
    done = False
    action_count = 0
    total_reward = 0
    while not done and action_count < 500:
        action = model.predict(np.array([observation]), verbose=0)

        # action = number with the highest probability
        action = np.argmax(action)
        #print(action)

        observation, reward, truncted, terminated, info = env.step(action)
        action_count += 1
        done = terminated or truncted
        total_reward += reward

    return total_reward

In [ ]:
#start learning and rendering the game
population_size = 10
p_crossover = 0.7
p_mutation = 0.5
best_weights = run_ga(pop_size=population_size, gen_limit=10,
                      p_crossover=p_crossover, p_mutation=p_mutation)


Creating initial population...
Evaluating initial population...


In [ ]:
#show the game with best weights
best_weights

In [18]:
time.sleep(3)
# play with the best weights
env = gym.make('CartPole-v1', render_mode='human')
first_layer_weights = np.array(best_weights[:4]).reshape(4, 1)
first_layer_bias = np.array(best_weights[4:5])
model.layers[0].set_weights([first_layer_weights, first_layer_bias])
observation = env.reset()[0]
done = False
action_count = 0
total_reward = 0
while not done and action_count < 500:
    action = model.predict(np.array([observation]), verbose=0)[0][0]
    action = round(action)
    #print('action  ', action)
    observation, reward, truncated, terminated, info = env.step(action)
    action_count += 1
    done = terminated or truncated
    total_reward += reward
    #time.sleep(0.1)
    env.render()

print(f"Total reward: {total_reward}")
env.close()
# THIS IS THE GOAL

Total reward: 500.0


In [19]:
env.close()